In [ ]:
# ===================================================================
# PHASE 5A: CORE CONVERSATIONAL AGENT
# Foundation for intelligent job search assistant
# ===================================================================
# Purpose: Natural language interface for job queries using Gemini AI
# Dependencies: google-generativeai, pandas, python-dotenv
# ===================================================================

import os
import json
import logging
from datetime import datetime
from google import genai
from google.genai import types
from typing import List, Dict, Optional, Any, Tuple
from dataclasses import dataclass, field
import pandas as pd
from enum import Enum

# ===================================================================
# LOGGING CONFIGURATION
# ===================================================================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler("conversational_agent.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("ConversationalAgent")

# ===================================================================
# INTENT CLASSIFICATION
# ===================================================================

class QueryIntent(Enum):
    """
    Enumeration of all possible user query intents.
    Helps classify what the user wants to do.
    """
    JOB_SEARCH = "job_search"              # "Find Python jobs"
    JOB_DETAILS = "job_details"            # "Tell me about TCS role"
    SKILL_ANALYSIS = "skill_analysis"      # "What skills do I need?"
    COMPARE_JOBS = "compare_jobs"          # "Compare these 2 jobs"
    APPLICATION_HELP = "application_help"  # "How do I apply?"
    SALARY_INFO = "salary_info"            # "What's the salary range?"
    LOCATION_QUERY = "location_query"      # "Jobs in Bangalore?"
    COMPANY_INFO = "company_info"          # "Tell me about Google"
    GENERAL_CHAT = "general_chat"          # "Hello", "Thanks"
    UNKNOWN = "unknown"                    # Unrecognized intent


# ===================================================================
# DATA CLASSES
# ===================================================================

@dataclass
class UserContext:
    """
    Maintains conversation context and user information.
    Essential for multi-turn conversations.
    """
    user_id: str
    conversation_history: List[Dict[str, str]] = field(default_factory=list)
    current_intent: Optional[QueryIntent] = None
    last_query: Optional[str] = None
    last_results: List[Dict] = field(default_factory=list)
    preferences: Dict[str, Any] = field(default_factory=dict)
    
    def add_message(self, role: str, content: str):
        """
        Add a message to conversation history.
        
        Args:
            role: 'user' or 'assistant'
            content: Message content
        """
        self.conversation_history.append({
            'role': role,
            'content': content,
            'timestamp': datetime.now().isoformat()
        })
        
        # Keep only last 10 messages for context window
        if len(self.conversation_history) > 10:
            self.conversation_history = self.conversation_history[-10:]
    
    def get_conversation_context(self) -> str:
        """
        Get formatted conversation history for LLM context.
        
        Returns:
            Formatted conversation string
        """
        context = ""
        for msg in self.conversation_history[-5:]:  # Last 5 messages
            role = msg['role'].title()
            content = msg['content']
            context += f"{role}: {content}\n"
        return context


@dataclass
class EntityExtraction:
    """
    Extracted entities from user query.
    Structured information parsed from natural language.
    """
    companies: List[str] = field(default_factory=list)
    skills: List[str] = field(default_factory=list)
    locations: List[str] = field(default_factory=list)
    positions: List[str] = field(default_factory=list)
    salary_min: Optional[float] = None
    salary_max: Optional[float] = None
    experience_years: Optional[int] = None
    work_mode: Optional[str] = None  # Remote, Hybrid, On-site
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for logging/debugging."""
        return {
            'companies': self.companies,
            'skills': self.skills,
            'locations': self.locations,
            'positions': self.positions,
            'salary_min': self.salary_min,
            'salary_max': self.salary_max,
            'experience_years': self.experience_years,
            'work_mode': self.work_mode
        }


# ===================================================================
# GEMINI API INTEGRATION
# ===================================================================

class GeminiAgent:
    """
    Core agent using Google Gemini for natural language understanding.
    Handles all LLM interactions and response generation.
    """
    
class GeminiAgent:
    """
    Core agent using Google Gemini for natural language understanding.
    Handles all LLM interactions and response generation.
    """
    def __init__(self, api_key: Optional[str] = None):
        """
        Initialize Gemini agent with the new Client.
        
        Args:
            api_key: Google Gemini API key (or from environment variable)
        """
        self.logger = logging.getLogger("GeminiAgent")
        
        # Check for  library
        if not genai:
            self.logger.error("❌ google-genai not installed!")
            self.logger.error("Install with: pip install google-genai")
            raise ImportError("Google Gen AI SDK not found")
        
        # Get API key
        self.api_key = api_key or os.getenv('GEMINI_API_KEY')
        if not self.api_key:
            self.logger.error("❌ GEMINI_API_KEY not found!")
            self.logger.error("Set environment variable or pass api_key parameter")
            raise ValueError("Gemini API key required")
        
        # ---Initialize Client ---
        try:
            self.client = genai.Client(api_key=self.api_key)
            self.model_name = "gemini-2.5-flash"
            self.logger.info("✅ Gemini Client initialized successfully")
        except Exception as e:
            self.logger.error(f"❌ Failed to initialize Gemini Client: {e}")
            raise
        
        # System prompt configuration
        self.system_prompt = """You are an intelligent job search assistant helping users find relevant job opportunities.

Your capabilities:
1. Search and filter jobs based on natural language queries
2. Provide detailed job information
3. Analyze skill requirements and gaps
4. Compare different job opportunities
5. Give application advice

Guidelines:
- Be concise but informative
- Use a friendly, professional tone
- Focus on helping users make informed decisions
- Always reference specific data when available
- Admit when you don't have information

When analyzing queries:
- Extract key entities (companies, skills, locations, salary, etc.)
- Identify user intent clearly
- Consider conversation context
"""
    
    def classify_intent(self, query: str, context: UserContext) -> QueryIntent:
        """
        Classify user query intent using Gemini.
        
        Args:
            query: User's natural language query
            context: Conversation context
            
        Returns:
            Classified QueryIntent
        """
        try:
            # Create intent classification prompt
            intent_prompt = f"""Classify the following user query into ONE of these intents:

            Intents:
            - job_search: User wants to find/search for jobs
            - job_details: User wants details about a specific job
            - skill_analysis: User asking about skills/requirements
            - compare_jobs: User wants to compare multiple jobs
            - application_help: User needs help with application process
            - salary_info: User asking about salary/compensation
            - location_query: User asking about job locations
            - company_info: User asking about a company
            - general_chat: Greetings, thanks, or casual conversation
            - unknown: Cannot determine intent

            User Query: "{query}"

            Recent Context:
            {context.get_conversation_context()}

            Respond with ONLY the intent name (e.g., "job_search"). No explanation."""

            # Get response from Gemini
            response = self.client.models.generate_content(
                model=self.model_name,
                contents=intent_prompt,
                config=types.GenerateContentConfig(
                    temperature=0.4  # Low temperature for deterministic classification
                )
            )
            intent_str = response.text.strip().lower()
            
            # Map to QueryIntent enum
            intent_mapping = {
                'job_search': QueryIntent.JOB_SEARCH,
                'job_details': QueryIntent.JOB_DETAILS,
                'skill_analysis': QueryIntent.SKILL_ANALYSIS,
                'compare_jobs': QueryIntent.COMPARE_JOBS,
                'application_help': QueryIntent.APPLICATION_HELP,
                'salary_info': QueryIntent.SALARY_INFO,
                'location_query': QueryIntent.LOCATION_QUERY,
                'company_info': QueryIntent.COMPANY_INFO,
                'general_chat': QueryIntent.GENERAL_CHAT,
                'unknown': QueryIntent.UNKNOWN
            }
            
            intent = intent_mapping.get(intent_str, QueryIntent.UNKNOWN)
            self.logger.info(f"📌 Intent classified: {intent.value}")
            
            return intent
            
        except Exception as e:
            self.logger.error(f"Error classifying intent: {e}")
            return QueryIntent.UNKNOWN
    
    def extract_entities(self, query: str) -> EntityExtraction:
        """
        Extract structured entities from natural language query.
        
        Args:
            query: User's natural language query
            
        Returns:
            EntityExtraction object with parsed entities
        """
        try:
            # Create entity extraction prompt
            extraction_prompt = f"""Extract structured information from this job search query.

            Query: "{query}"

            Extract and return a JSON object with these fields (use null for missing values):
            {{
            "companies": ["company names"],
            "skills": ["technical skills"],
            "locations": ["cities/locations"],
            "positions": ["job titles/roles"],
            "salary_min": number or null (in LPA),
            "salary_max": number or null (in LPA),
            "experience_years": number or null,
            "work_mode": "Remote" or "Hybrid" or "On-site" or null
            }}

            Examples:
            Query: "Python developer jobs in Bangalore with 5 LPA"
            Result: {{"skills": ["python"], "locations": ["bangalore"], "positions": ["developer"], "salary_min": 5}}

            Query: "Remote data scientist role at Google with 3 years experience"
            Result: {{"companies": ["google"], "positions": ["data scientist"], "work_mode": "Remote", "experience_years": 3}}

            Return ONLY valid JSON, no other text."""

            # Get response from Gemini
            response = self.client.models.generate_content(
                model=self.model_name,
                contents=extraction_prompt,
                config=types.GenerateContentConfig(
                    response_mime_type="application/json",
                    response_schema={
                        "type": "object",
                        "properties": {
                            "companies": {"type": "array", "items": {"type": "string"}},
                            "skills": {"type": "array", "items": {"type": "string"}},
                            "locations": {"type": "array", "items": {"type": "string"}},
                            "positions": {"type": "array", "items": {"type": "string"}},
                            "salary_min": {"type": "number"},
                            "salary_max": {"type": "number"},
                            "experience_years": {"type": "number"},
                            "work_mode": {"type": "string"}
                        }
                    }
                )
            )

            result_text = response.text.strip()
            
            # Parse JSON
            extracted_data = json.loads(result_text)
            
            # Create EntityExtraction object
            entities = EntityExtraction(
                companies=extracted_data.get('companies', []) or [],
                skills=extracted_data.get('skills', []) or [],
                locations=extracted_data.get('locations', []) or [],
                positions=extracted_data.get('positions', []) or [],
                salary_min=extracted_data.get('salary_min'),
                salary_max=extracted_data.get('salary_max'),
                experience_years=extracted_data.get('experience_years'),
                work_mode=extracted_data.get('work_mode')
            )
            
            self.logger.info(f"📊 Entities extracted: {entities.to_dict()}")
            
            return entities
            
        except Exception as e:
            self.logger.error(f"Error extracting entities: {e}")
            return EntityExtraction()
    
    def generate_response(
        self,
        query: str,
        context: UserContext,
        data: Optional[Dict[str, Any]] = None
    ) -> str:
        """
        Generate natural language response using Gemini.
        
        Args:
            query: User query
            context: Conversation context
            data: Retrieved data to include in response
            
        Returns:
            Generated response text
        """
        try:
            # Build response prompt
            response_prompt = f"""You are a job search assistant. Generate a helpful response.

            User Query: "{query}"

            Conversation Context:
            {context.get_conversation_context()}

            Available Data:
            {json.dumps(data, indent=2) if data else "No specific data available"}

            Guidelines:
            - Be conversational and helpful
            - Reference specific data when available
            - Keep response concise (2-4 paragraphs max)
            - Use formatting like bullet points when appropriate
            - End with a helpful follow-up question or suggestion

            Generate response:"""

            # Get response from Gemini
            response = self.client.models.generate_content(
                model=self.model_name,
                contents=response_prompt,
                config=types.GenerateContentConfig(
                    system_instruction=self.system_prompt, 
                    temperature=0.7
                )
            )
            generated_text = response.text.strip()
            
            self.logger.info(f"💬 Response generated ({len(generated_text)} chars)")
            
            return generated_text
            
        except Exception as e:
            self.logger.error(f"Error generating response: {e}")
            return "I apologize, but I'm having trouble generating a response right now. Could you please rephrase your question?"


# ===================================================================
# CONVERSATIONAL AGENT ORCHESTRATOR
# ===================================================================

class ConversationalAgent:
    """
    Main agent orchestrator that coordinates all components.
    This is the primary interface for the job search assistant.
    """
    
    def __init__(
        self,
        gemini_api_key: Optional[str] = None,
        jobs_csv_path: str = None
    ):
        """
        Initialize the conversational agent.
        
        Args:
            gemini_api_key: Gemini API key
            jobs_csv_path: Path to prioritized jobs CSV
        """
        self.logger = logging.getLogger("ConversationalAgent")
        
        self.logger.info("="*70)
        self.logger.info("🤖 INITIALIZING CONVERSATIONAL AGENT")
        self.logger.info("="*70)
        
        # Initialize Gemini agent
        try:
            self.gemini = GeminiAgent(api_key=gemini_api_key)
            self.logger.info("✅ Gemini agent initialized")
        except Exception as e:
            self.logger.error(f"❌ Failed to initialize Gemini: {e}")
            raise
        
        # Load jobs data
        try:
            self.jobs_df = pd.read_csv(jobs_csv_path)
            self.logger.info(f"✅ Loaded {len(self.jobs_df)} jobs from {jobs_csv_path}")
        except Exception as e:
            self.logger.error(f"❌ Failed to load jobs data: {e}")
            raise
        
        # User contexts (in production, store in database)
        self.user_contexts: Dict[str, UserContext] = {}
        
        self.logger.info("="*70)
        self.logger.info("✅ AGENT READY FOR CONVERSATIONS")
        self.logger.info("="*70 + "\n")
    
    def get_or_create_context(self, user_id: str) -> UserContext:
        """
        Get existing user context or create new one.
        
        Args:
            user_id: Unique user identifier
            
        Returns:
            UserContext object
        """
        if user_id not in self.user_contexts:
            self.user_contexts[user_id] = UserContext(user_id=user_id)
            self.logger.info(f"👤 Created new context for user: {user_id}")
        return self.user_contexts[user_id]
    
    def process_query(
        self,
        query: str,
        user_id: str = "default_user"
    ) -> Dict[str, Any]:
        """
        Process a user query and return structured response.
        
        Args:
            query: User's natural language query
            user_id: User identifier
            
        Returns:
            Dictionary with response data
        """
        self.logger.info(f"\n{'='*70}")
        self.logger.info(f"📥 NEW QUERY from {user_id}")
        self.logger.info(f"Query: {query}")
        self.logger.info(f"{'='*70}")
        
        # Get user context
        context = self.get_or_create_context(user_id)
        context.add_message('user', query)
        context.last_query = query
        
        try:
            # Step 1: Classify intent
            intent = self.gemini.classify_intent(query, context)
            context.current_intent = intent
            
            # Step 2: Extract entities
            entities = self.gemini.extract_entities(query)
            
            # Step 3: Retrieve relevant data (placeholder for now)
            data = self._retrieve_data(intent, entities)
            
            # Step 4: Generate response
            response_text = self.gemini.generate_response(query, context, data)
            
            # Add response to context
            context.add_message('assistant', response_text)
            
            # Prepare response
            response = {
                'status': 'success',
                'query': query,
                'intent': intent.value,
                'entities': entities.to_dict(),
                'response': response_text,
                'data': data,
                'timestamp': datetime.now().isoformat()
            }
            
            self.logger.info(f"✅ Query processed successfully")
            self.logger.info(f"{'='*70}\n")
            
            return response
            
        except Exception as e:
            self.logger.error(f"❌ Error processing query: {e}")
            
            error_response = {
                'status': 'error',
                'query': query,
                'error': str(e),
                'response': "I apologize, but I encountered an error processing your request. Please try again.",
                'timestamp': datetime.now().isoformat()
            }
            
            return error_response
    
    def _retrieve_data(
        self,
        intent: QueryIntent,
        entities: EntityExtraction
    ) -> Dict[str, Any]:
        """
        Retrieve relevant data based on intent and entities.
        This is a placeholder - will be enhanced with RAG system.
        
        Args:
            intent: Classified intent
            entities: Extracted entities
            
        Returns:
            Retrieved data dictionary
        """
        # For now, return basic stats
        # Will be replaced with actual retrieval in Phase 5B
        
        data = {
            'total_jobs': len(self.jobs_df),
            'intent': intent.value,
            'filtered_count': 0,
            'jobs': []
        }
        
        # Simple filtering based on entities (basic implementation)
        filtered_df = self.jobs_df.copy()
        
        # Filter by skills
        if entities.skills:
            skill_filter = filtered_df['skills_required'].str.contains(
                '|'.join(entities.skills),
                case=False,
                na=False
            )
            filtered_df = filtered_df[skill_filter]
        
        # Filter by locations
        if entities.locations:
            location_filter = filtered_df['location_city'].str.contains(
                '|'.join(entities.locations),
                case=False,
                na=False
            )
            filtered_df = filtered_df[location_filter]
        
        # Filter by companies
        if entities.companies:
            company_filter = filtered_df['company_name'].str.contains(
                '|'.join(entities.companies),
                case=False,
                na=False
            )
            filtered_df = filtered_df[company_filter]
        
        data['filtered_count'] = len(filtered_df)
        
        # Get top 5 results
        if len(filtered_df) > 0:
            top_jobs = filtered_df.head(5)
            data['jobs'] = top_jobs[[
                'job_id', 'company_name', 'position_title',
                'location_city', 'salary_max', 'final_priority_score'
            ]].to_dict('records')
        
        return data


# ===================================================================
# TESTING & EXAMPLE USAGE
# ===================================================================

def test_agent():
    """
    Test the conversational agent with example queries.
    """
    logger.info("\n" + "="*70)
    logger.info("🧪 TESTING CONVERSATIONAL AGENT")
    logger.info("="*70 + "\n")
    
    # Initialize agent
    try:
        agent = ConversationalAgent(
            gemini_api_key=os.getenv('GEMINI_API_KEY'),
            jobs_csv_path=None
        )
    except Exception as e:
        logger.error(f"Failed to initialize agent: {e}")
        return
    
    # Test queries
    test_queries = [
        "Hello! Can you help me find a job?",
        "Show me Python developer jobs in Bangalore",
        "What are the salary ranges for data scientist positions?",
        "Tell me about remote work opportunities"
    ]
    
    for query in test_queries:
        logger.info(f"\n{'─'*70}")
        logger.info(f"Testing query: {query}")
        logger.info(f"{'─'*70}")
        
        response = agent.process_query(query, user_id="test_user")
        
        logger.info(f"\nIntent: {response.get('intent')}")
        logger.info(f"Entities: {response.get('entities')}")
        logger.info(f"\nResponse:\n{response.get('response')}\n")


# ===================================================================
# MAIN EXECUTION
# ===================================================================

if __name__ == "__main__":
    # Load API key from .env file
    from dotenv import load_dotenv
    load_dotenv()  # Loads GEMINI_API_KEY from .env file
    
    # Verify API key is set
    if not os.getenv('GEMINI_API_KEY'):
        print('Error: Set GEMINI_API_KEY in .env file')

    # Run tests
    test_agent()
    
    # Interactive mode (optional)
    print("\n" + "="*70)
    print("💬 INTERACTIVE MODE")
    print("="*70)
    print("Type your questions (or 'quit' to exit)\n")
    
    try:
        csv_path = r"D:\Projects By Month\November 2025\Placement Mail Analysis System\.venv\Phase_scripts\Phase 4\prioritized_jobs.csv"
        agent = ConversationalAgent(
            gemini_api_key=os.getenv('GEMINI_API_KEY'),
            jobs_csv_path=csv_path
        )
        
        while True:
            user_input = input("\n🧑 You: ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("\n👋 Goodbye! Have a great day!")
                break
            
            if not user_input:
                continue
            
            response = agent.process_query(user_input)
            print(f"\n🤖 Assistant: {response['response']}")
            
    except KeyboardInterrupt:
        print("\n\n👋 Goodbye!")
    except Exception as e:
        logger.error(f"Error in interactive mode: {e}")